In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

In [20]:
#load the data
data = pd.read_csv(r'C:\Users\Wahaj Sajid\Desktop\Datasets\heart_disease_data\framingham.csv')
data_test = pd.read_csv(r'C:\Users\Wahaj Sajid\Desktop\Datasets\heart_disease_data\framingham.csv')

In [21]:
data_test.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [22]:
data

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0


In [23]:
print(data.isnull().sum().sort_values(ascending=False))

glucose            388
education          105
BPMeds              53
totChol             50
cigsPerDay          29
BMI                 19
heartRate            1
male                 0
age                  0
currentSmoker        0
prevalentStroke      0
prevalentHyp         0
diabetes             0
sysBP                0
diaBP                0
TenYearCHD           0
dtype: int64


In [24]:
data = data.drop('education', axis =1)

In [25]:
#imputing the missing values
imputer = SimpleImputer(strategy='mean')
data['glucose'] = imputer.fit_transform(data[['glucose']])
data['BPMeds'] = imputer.fit_transform(data[['BPMeds']])
data['totChol'] = imputer.fit_transform(data[['totChol']])
data['cigsPerDay'] = imputer.fit_transform(data[['cigsPerDay']])
data['BMI'] = imputer.fit_transform(data[['BMI']])
data['heartRate'] = imputer.fit_transform(data[['heartRate']])

In [26]:
#scaling the data
x_scaler = StandardScaler()
data['age'] = x_scaler.fit_transform(data[['age']])
data['cigsPerDay'] = x_scaler.fit_transform(data[['cigsPerDay']])
data['totChol'] = x_scaler.fit_transform(data[['totChol']])
data['sysBP'] = x_scaler.fit_transform(data[['sysBP']])
data['diaBP'] = x_scaler.fit_transform(data[['diaBP']])
data['BMI'] = x_scaler.fit_transform(data[['BMI']])
data['heartRate'] = x_scaler.fit_transform(data[['heartRate']])
data['glucose'] = x_scaler.fit_transform(data[['glucose']])

In [27]:
data['BPMeds'] = data['BPMeds'].astype(int)

In [28]:
#assigning x and y
x = data.drop('TenYearCHD', axis=1)
y = data['TenYearCHD']

In [29]:
#train test split the model
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.8)

In [30]:
#train the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape = (x_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stop = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True )
model.compile(loss = BinaryCrossentropy(), optimizer='adam', metrics=['mae'])
model.fit(x, y,epochs = 50, batch_size=16, callbacks= [early_stop])


Epoch 1/50


c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5415 - mae: 0.3875
Epoch 2/50
 83/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3912 - mae: 0.2510

c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)


265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4069 - mae: 0.2576
Epoch 3/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3912 - mae: 0.2392
Epoch 4/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3954 - mae: 0.2424
Epoch 5/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3997 - mae: 0.2454
Epoch 6/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3848 - mae: 0.2331
Epoch 7/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3855 - mae: 0.2384
Epoch 8/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3803 - mae: 0.2348
Epoch 9/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3733 - mae: 0.2313
Epoch 10/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3670 - mae: 0.2295
Epoch 11/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3769 - mae: 0.2328
Epoch 12/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3636 - mae: 0.2239
Epoch 13/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3699 - mae: 0.2247
Epoch 14/50
265/265 ━━━━━━━━━━━━━━━

In [31]:
loss, mae  = model.evaluate(x_test, y_test)
print(f"Loss: ",loss)
print(f"mae: ",mae)

106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3388 - mae: 0.2157
Loss:  0.3431718051433563
mae:  0.21597903966903687


In [32]:
#predict the model
age = 46
cigsPerDay = 0.0
totChol = 250
sysBP = 121
diaBP = 81
BMI = 28.34
hearRate = 95
glucose = 76

age = x_scaler.transform([[age]])[0][0]
cigsPerDay = x_scaler.transform([[cigsPerDay]])[0][0]
totChol = x_scaler.transform([[totChol]])[0][0]
sysBP = x_scaler.transform([[sysBP]])[0][0]
diaBP = x_scaler.transform([[diaBP]])[0][0]
BMI = x_scaler.transform([[BMI]])[0][0]
hearRate = x_scaler.transform([[hearRate]])[0][0]
glucose = x_scaler.transform([[glucose]])[0][0]

test_features = np.array([[1,age,0,0,0,0,0,0,totChol,sysBP,diaBP,BMI,hearRate,glucose]])

prediction = model.predict(test_features)
print(prediction[0][0])
if(prediction>=0.5):
    print("The person is likely to develop heart disease in next 10 years")
else:
    print("The person is not likely to develop heart disease in next 10 years")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
0.0794991
The person is not likely to develop heart disease in next 10 years


c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Wahaj Sajid\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Us